## Import libraries

In [1]:
import pandas as pd
import urllib.request
import zipfile


### Download the results file from the TSE Dados Abertos

Uncomment when running the code for the first time, then you can comment again

In [2]:

# url_download = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2022_BR.zip'

# opener = urllib.request.build_opener()
# opener.addheaders = [('User-agent', 'Mozilla/5.0')]
# urllib.request.install_opener(opener)
# urllib.request.urlretrieve(url_download, f"CSV/votacao_secao_2022_BR.zip")

If you prefer you can download the files manually: The full list of files can be found [here](https://dadosabertos.tse.jus.br/dataset/resultados-2022)!

### Unzip the downloaded file


Uncomment when running the code for the first time, then you can comment again

In [3]:
# with zipfile.ZipFile(f"CSV/votacao_secao_2022_BR.zip", 'r') as zip_ref:
#     zip_ref.extractall("CSV/")

### Read the extracted CSV

In [4]:
df = pd.read_csv("CSV/votacao_secao_2022_BR.csv", encoding="Latin 1", sep=";", usecols=['SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','DS_CARGO','NM_VOTAVEL','QT_VOTOS'])

### Quick check of a specific Secao, Zona, City and State

First define a state, city, zona and secao

In [5]:
uf = 'SP'
city = 'SÃO PAULO'
zona = 5
secao = 62

In [6]:
check_secao = df[(df['NR_ZONA']==zona)&(df['NR_SECAO']==secao)&(df['SG_UF']==uf)&(df['NM_MUNICIPIO']==city)][['NM_VOTAVEL','QT_VOTOS']].groupby(by='NM_VOTAVEL').sum("QT_VOTOS")
check_secao

,QT_VOTOS
NM_VOTAVEL,
CIRO FERREIRA GOMES,9
JAIR MESSIAS BOLSONARO,133
LUIZ FELIPE CHAVES D AVILA,5
LUIZ INÁCIO LULA DA SILVA,94
SIMONE NASSAR TEBET,44
SOFIA PADUA MANZANO,1
VOTO BRANCO,4
VOTO NULO,7


### Extractor of votes by Zona and Secao for a specific municipality and aggregate them by zone and secao

Need to define city and state and then run the code

In [7]:
uf = 'SP'
city = 'SÃO PAULO'

In [8]:
df_filtered = df[(df['SG_UF']==uf)&(df['NM_MUNICIPIO']==city)]
df_filtered = df_filtered[(df_filtered['NM_VOTAVEL']!='VOTO NULO')&(df_filtered['NM_VOTAVEL']!='VOTO BRANCO')]
df_results = df_filtered[['SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO']].drop_duplicates()
df_results['Lula_abslt'] = ""
df_results['Bolsonaro_abslt'] = ""
df_results['Lula_pctg'] = ""
df_results['Bolsonaro_pctg'] = ""
df_results['VotosTotais'] = ""

for i,r in df_results.iterrows():
    #print(r['NR_ZONA'],r['NR_SECAO'])
    zona = r['NR_ZONA']
    secao = r['NR_SECAO']
    results = df_filtered[(df_filtered['NR_ZONA']==zona)&(df_filtered['NR_SECAO']==secao)][['NM_VOTAVEL','QT_VOTOS']].groupby(by='NM_VOTAVEL').sum("QT_VOTOS")
    results['pctg'] = (results.QT_VOTOS / results.QT_VOTOS.sum())*100
    df_results.at[i,'Lula_pctg'] = results[results.index=='LUIZ INÁCIO LULA DA SILVA']['pctg'][0]
    df_results.at[i,'Bolsonaro_pctg'] = results[results.index=='JAIR MESSIAS BOLSONARO']['pctg'][0]
    df_results.at[i,'Lula_abslt'] = results[results.index=='LUIZ INÁCIO LULA DA SILVA']['QT_VOTOS'][0]
    df_results.at[i,'Bolsonaro_abslt'] = results[results.index=='JAIR MESSIAS BOLSONARO']['QT_VOTOS'][0]
    df_results.at[i,'VotosTotais'] = results.QT_VOTOS.sum()



In [9]:
df_results

,SG_UF,NM_MUNICIPIO,NR_ZONA,NR_SECAO,Lula_abslt,Bolsonaro_abslt,Lula_pctg,Bolsonaro_pctg,VotosTotais
7,SP,SÃO PAULO,374,173,128,94,47.407407,34.814815,270
79,SP,SÃO PAULO,389,182,114,91,47.89916,38.235294,238
85,SP,SÃO PAULO,381,75,94,63,53.714286,36.0,175
87,SP,SÃO PAULO,353,383,114,79,52.054795,36.073059,219
110,SP,SÃO PAULO,418,117,157,98,52.684564,32.885906,298
...,...,...,...,...,...,...,...,...,...
2015795,SP,SÃO PAULO,326,718,114,113,44.88189,44.488189,254
2016377,SP,SÃO PAULO,252,38,104,126,37.818182,45.818182,275
2016989,SP,SÃO PAULO,248,46,125,93,48.449612,36.046512,258
2016996,SP,SÃO PAULO,422,93,152,96,55.677656,35.164835,273


In [10]:
df_filtered['QT_VOTOS'].sum()

6892638

In [11]:
df_results['VotosTotais'].sum()

6892638

In [12]:
df_presidente = df[df['DS_CARGO']=='PRESIDENTE']
df_presidente

,SG_UF,NM_MUNICIPIO,NR_ZONA,NR_SECAO,DS_CARGO,NM_VOTAVEL,QT_VOTOS
0,BA,EUNÁPOLIS,188,319,PRESIDENTE,SIMONE NASSAR TEBET,6
1,MA,CHAPADINHA,42,169,PRESIDENTE,SIMONE NASSAR TEBET,1
2,PR,SANTO ANTÔNIO DA PLATINA,22,93,PRESIDENTE,SIMONE NASSAR TEBET,8
3,TO,CRISTALÂNDIA,13,71,PRESIDENTE,SIMONE NASSAR TEBET,9
4,RJ,CAMPOS DOS GOYTACAZES,129,101,PRESIDENTE,SIMONE NASSAR TEBET,20
...,...,...,...,...,...,...,...
3529839,PB,PRINCESA ISABEL,34,233,PRESIDENTE,VOTO BRANCO,4
3529840,RN,BARAÚNA,58,104,PRESIDENTE,VOTO BRANCO,3
3529841,RN,MOSSORÓ,34,242,PRESIDENTE,VOTO BRANCO,7
3529842,PI,PICOS,10,354,PRESIDENTE,VOTO BRANCO,2


In [13]:
# df_presidente[df_presidente['SG_UF']=='SP'].groupby(by=['NM_MUNICIPIO','NM_VOTAVEL']).sum("QT_VOTOS").to_clipboard()

In [14]:
df_filtered['NM_VOTAVEL'].unique()

array(['SIMONE NASSAR TEBET', 'JAIR MESSIAS BOLSONARO',
       'KELMON LUIS DA SILVA SOUZA', 'LUIZ INÁCIO LULA DA SILVA',
       'CIRO FERREIRA GOMES', 'VERA LUCIA PEREIRA DA SILVA SALGADO',
       'SORAYA VIEIRA THRONICKE', 'LUIZ FELIPE CHAVES D AVILA',
       'SOFIA PADUA MANZANO', 'LEONARDO PÉRICLES VIEIRA ROQUE',
       'JOSE MARIA EYMAEL'], dtype=object)

### Import Zonas Eleitorais with addresses

Import the file corresponding to the city you are analyzing

In [15]:
df_zonas_add = pd.read_csv("Output/Zonas and Secoes Sao Paulo NORMALIZED.csv")

In [16]:
df_zonas_add

,Unnamed: 0,Zona,Local,Endereco,Secoes,Cidade
0,0,1,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207",460,SÃO PAULO
1,1,1,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207",421,SÃO PAULO
2,2,1,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207",97,SÃO PAULO
3,3,1,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207",87,SÃO PAULO
4,4,1,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207",393,SÃO PAULO
...,...,...,...,...,...,...
16416,16416,422,EMEI. ROSA CAROLINA AGAZZI,"RUA DOUTOR ARAÚJO CASTRO, S/N",247,SÃO PAULO
16417,16417,422,EMEI. SAVA POPOVIC,"RUA FRIBURGO, 12",323,SÃO PAULO
16418,16418,422,EMEI. SAVA POPOVIC,"RUA FRIBURGO, 12",336,SÃO PAULO
16419,16419,422,EMEI. SAVA POPOVIC,"RUA FRIBURGO, 12",294,SÃO PAULO


In [17]:
vote_counts_address = pd.merge(df_zonas_add, df_results[['NR_ZONA','NR_SECAO','Lula_abslt','Bolsonaro_abslt','Lula_pctg','Bolsonaro_pctg','VotosTotais']],  how='right', left_on=['Zona','Secoes'], right_on = ['NR_ZONA','NR_SECAO'])

In [18]:
vote_counts_address.drop(columns=['Unnamed: 0'], inplace=True)

In [19]:
vote_counts_address

,Zona,Local,Endereco,Secoes,Cidade,NR_ZONA,NR_SECAO,Lula_abslt,Bolsonaro_abslt,Lula_pctg,Bolsonaro_pctg,VotosTotais
0,374.0,E M E F BRASIL JAPÃO,"RUA DOUTOR PAULO CARVALHO FERREIRA, 94",173.0,SÃO PAULO,374,173,128,94,47.407407,34.814815,270
1,NaN,NaN,NaN,NaN,NaN,389,182,114,91,47.89916,38.235294,238
2,NaN,NaN,NaN,NaN,NaN,381,75,94,63,53.714286,36.0,175
3,353.0,EE PROFº SATURNINO PEREIRA,"RUA BERNARDINO ANTUNES, 200",383.0,SÃO PAULO,353,383,114,79,52.054795,36.073059,219
4,418.0,EE. MANUEL BORBA GATO,"ESTRADA DO ALVARENGA, 179",117.0,SÃO PAULO,418,117,157,98,52.684564,32.885906,298
...,...,...,...,...,...,...,...,...,...,...,...,...
26283,326.0,E.M.E.I. RODRIGO SOARES JÚNIOR,"RUA CAIÇARA DO RIO DO VENTO, 237",718.0,SÃO PAULO,326,718,114,113,44.88189,44.488189,254
26284,252.0,EE. PADRE ANTÃO,"RUA SANTO AFONSO, 273",38.0,SÃO PAULO,252,38,104,126,37.818182,45.818182,275
26285,NaN,NaN,NaN,NaN,NaN,248,46,125,93,48.449612,36.046512,258
26286,422.0,E.E. PROFª ELZA SARAIVA MONTEIRO,"AVENIDA DONA MARIANA CALIGIORI RONCHETTI, 1200",93.0,SÃO PAULO,422,93,152,96,55.677656,35.164835,273


In [20]:
vote_counts_address["VotosTotais"].sum()

6892638

## Possible analysis:

### 1) By name of zona

In [21]:
# local = 'EEEF PROF. LANGENDONCK'
# apuracao = vote_counts_address[(vote_counts_address['Local']==local)]
# votos_lula = apuracao['Lula_abslt'].sum()
# votos_bolsonaro = apuracao['Bolsonaro_abslt'].sum()
# votos_totais = apuracao['VotosTotais'].sum()
# print(f"\nNome Secao : {local} ")
# print(f"Votos totais na secao : {votos_totais} \n")
# print(f"Votos totais Lula na secao : {votos_lula} | Percentual: {(votos_lula/votos_totais)*100}%")
# print(f"Votos totais Bolsonaro na secao : {votos_bolsonaro} | Percentual: {(votos_bolsonaro/votos_totais)*100}%")
# print(f"Votos totais OUTROS (outros candidatos/branco/nulo) na secao : {votos_totais-votos_bolsonaro-votos_lula} | Percentual: {((votos_totais-votos_bolsonaro-votos_lula)/votos_totais)*100}%")


### 2) Places where Bolsonaro had less votes than 11%

In [22]:
bolsonaro_less_11pct = vote_counts_address[vote_counts_address['Bolsonaro_pctg']<11]
contagem = bolsonaro_less_11pct['Local'].count()
print(f"Em quantos locais bolsonaro teve menos de 11% dos votos: {contagem}")
print(f"Total de locais de votacao: {vote_counts_address['Local'].count()}")
print(f"Percentual de locais de votacao on Bolsonaro teve menos de 11%: {(contagem/vote_counts_address['Local'].count())*100}%")


Em quantos locais bolsonaro teve menos de 11% dos votos: 11
Total de locais de votacao: 16361
Percentual de locais de votacao on Bolsonaro teve menos de 11%: 0.06723305421429009%


In [23]:
print("Locais onde Bolsonaro teve menos de 11% dos votos:\n")
for i in bolsonaro_less_11pct['Local']:
    print(i)

Locais onde Bolsonaro teve menos de 11% dos votos:

FUNDAÇÃO CASA - SÃO PAULO
FUNDAÇÃO CASA-JD SÂO LUIZ-CASA II
UNID. DE INTERNAÇÃO - OURO PRETO E JOÃO DO PULO (F.CASA)
CDP DE VILA INDEPENDÊNCIA
PENITENCIÁRIA FEMININA SANTANA
CDP IV CENTRO DE DETENÇÃO PROVISÓRIA DE PINHEIROS IV
UNIDADE DE INTERNAÇÃO - PAULISTA E NOVA VIDA (F.CASA)
CDP III  CENTRO DE DETENÇÃO PROVISÓRIA DE PINHEIROS III
FUNDAÇÃO CASA - UNIDADE DE INTERNAÇÃO PROVISÓRIA - COMPLEXO BRÁS
CDP II CENTRO DE DETENÇÃO PROVISÓRIA DE PINHEIROS II
UNIDADE INTERNAÇÃO - NOVA AROEIRA (UI-37)


In [24]:
vote_counts_address.to_excel("Output/VotesSaoPaulo.xlsx")